In [237]:
from dotenv import load_dotenv
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = 'True'

load_dotenv()

True

In [238]:
!where python

d:\Hackathon\venv\python.exe
C:\Users\HARIHARAN\anaconda3\python.exe
C:\Users\HARIHARAN\AppData\Local\Microsoft\WindowsApps\python.exe


In [239]:
#pip install langchain_ollama langchain_core pymysql

In [240]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool

In [241]:
llm = ChatOllama(model='sqlcoder', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content='].\n 1015 [[48326,17910,17907,17906]=>[[48326,17910,17907,17906],[1015]] .\n 17906 [[17910,48326,1015,77277]=>[[17910,48326,1015,77277],[17906]] .\n 48326 => [[1015,17907,17906,77277,17910],[48326]] .\n 77277 => [[48326,17907,17906,1015,17910],[77277]] .\n 77278 => [[48327,77277]=>[[48327,77277],[77278]] .\n 77279 => [[48326,17907,17906,1015,17910],[77279]] .\n 77280 =>/ [[17907,17906,1015,48326]=>[[17907,17906,1015,48326],[77280]] .\n 77281 => [[17907,17906,1015,48326],[77281]] .\n 77282 => [[17907,17906,1015,48326],[77282]] .\n 77283 =>[[17907,17906,1015,48326],[77283]] .\n 77284 =>/ [[17907,17906,1015,48326],[77284]] .\n 77285 =>[[17907,17906,1015,48326],[77285]] .\n 77286 => [[17907,17906,1015,48326],[77286]] .\n 77287 =>/ [[17907,17906,1015,48326],[77287]] .\n 77288 =>[[17907,17906,1015,48326],[77288]] .\n 77289 => [[17907,17906,1015,48326],[77289]] .\n 77290 =>/ [[17907,17906,1015,48326],[77290]] .\n 77291 => [[17907,17906,1015,48326],[77291]] .\n 77292 =>/ [[1

In [288]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("mysql+pymysql://root@localhost/Studentdb",include_tables=["studentsperformance"])

In [289]:
db.dialect
db.get_usable_table_names()

['studentsperformance']

In [290]:
print(db.get_table_info())


CREATE TABLE studentsperformance (
	`Portal_ID` TEXT NOT NULL, 
	`Register_Number` TEXT, 
	`Name` TEXT, 
	`Department` TEXT, 
	`CTPS` FLOAT, 
	`L1` FLOAT, 
	`L2` FLOAT, 
	`L3` FLOAT, 
	`L4` FLOAT, 
	`L5` FLOAT, 
	`PDS` FLOAT, 
	`Total score` INTEGER, 
	`Date` DATE NOT NULL, 
	PRIMARY KEY (`Portal_ID`, `Date`)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from studentsperformance table:
Portal_ID	Register_Number	Name	Department	CTPS	L1	L2	L3	L4	L5	PDS	Total score	Date
AML_230001		ABINAYA S	AI & ML	90.05	100.0	40.0	0.0	0.0	0.0	54.0	8565	2024-08-13
AML_230002		ABINESH A	AI & ML	93.42	67.0	0.0	0.0	0.0	0.0	16.0	5681	2024-08-13
AML_230003		AKARSHANA S	AI & ML	94.0	96.0	7.0	0.0	0.0	0.0	62.0	8385	2024-08-28
*/


# SQL Query Chain

- Convert the question into Sql query
- execute the query;
- use the result to answer the original question

In [291]:
from langchain.chains import create_sql_query_chain

sql_chain = create_sql_query_chain(llm,db)

In [292]:
sql_chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [293]:
sql_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables

In [294]:
question = "how many students are there?"
response = sql_chain.invoke({'question':question})
print(response)

SELECT COUNT(*) FROM studentsperformance;


# Extract Correct Query from LLM

In [295]:
from scripts.llm import ask_llm

from langchain_core.runnables import chain

In [296]:
@chain

def get_correct_sql_query(input):
    context = input['context']
    question = input['question']

    instruction = """
        use above context to fetch the correct SQl query for the question
        {}

        Do not enclose query in ```sql and do not write preamble and explanation.
        You must return only single SQL query
        
    """.format(question)

    response = ask_llm(context=context, question=instruction, )

    return response
    

In [297]:
response=get_correct_sql_query.invoke({'context': response, 'question':question})
print(response)

SELECT COUNT(*) FROM studentsperformance;


In [298]:
db.run(response)

'[(111,)]'

# Final SQL Query Chain

In [299]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [300]:
execute_query = QuerySQLDataBaseTool(db=db)
sql_query = create_sql_query_chain(llm, db)

final_chain = (
    {'context': sql_query, 'question':RunnablePassthrough()}
     |get_correct_sql_query
    |execute_query
)

In [302]:
question = "how many students are there? "

response = final_chain.invoke({'question':question})
print(response)

[(111,)]


In [315]:
question = "Show me the students who scored more than 90 in PDS? "

response = final_chain.invoke({'question':question})
print(response)

[('ABINAYA S',), ('ABINESH A',), ('AKARSHANA S',), ('AKASH D',), ('ANUGRAHA S',), ('ARUNESHWARAN V S',), ('ASHIKA V U',), ('ATHULYA AM',), ('BHAVAN KUMAR K',), ('BOOPATHI M',), ('DEVANAND A',), ('GEO JEEVAN RAJ C',), ('GIRIPRIYAN S',), ('GOPIKA A S',), ('GOPIKA P',), ('GOWTHAM S T',), ('GOWTHAMAN D',), ('HARI HARAN P R',), ('HARISH V B',), ('HARISH T',), ('HARRINI D S',), ('HEMACHANDRAN S B',), ('HEMANTH KUMAR V',), ('ILAMATHI D',), ('INIYAN S K',), ('JAGADEESH R',), ('JAGADEESHWARAN -',), ('JAYASHREELAKSHMI S',), ('JAYASREE J',), ('JHEEVA SHANKAR M',), ('KAMALI K',), ('KANISHKA RAJ R',), ('KARTHIK S',), ('KISHORE V',), ('KOUSHIKA SHREE R',), ('LATESH K',), ('MABITHA M',), ('MADHU T',), ('MAHASHREE -',), ('MAHMUTHAL N',), ('MANIMARAN M',), ('MANIRAJA S',), ('MANOJ KUMAR M',), ('MATHANKUMAR S',), ('MEENA LOSINI T',), ('MITHRAJITH K S',), ('MOHAMED IRSATH M',), ('MOHAMMED SYFUDEEN -',), ('MONICA M',), ('MONICA P',), ('MONISHA V',), ('MUTHULAKSHMI M',), ('NAVA GEEVITHAN G',), ('NEHA S',),

# MySQL Query Execution with Agents with LangGraph

- it can answer questions based on the databases schema as well as on the databases content [like describing specific table]
- it can recove from errors by runnig a generated query , catching the traceback and regenerating it correctly
- it can query the database as many times as needed to answer the user queestion
-it will save tokens by only retrieving the schema from revelent tables

In [256]:
#%pip install --upgrade --quiet langgraph

In [303]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db ,llm=llm)

In [304]:
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002B99C048A10>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002B99C048A10>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000002B99C048A10>),
 QuerySQLCheckerTool(description='Use this tool to 

In [305]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
Important things

1.the table contains subjects [CTPs, L1,L2,L3,L4,L5,PDS]
2.completetion score of all subjects is greater than 90
3.if the user is asks 'how many ?' type of question just return a number don't limit anything

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

System_message = SystemMessage(content=SQL_PREFIX)

In [306]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier = System_message,debug=False)

In [308]:
question = "how many students are there?"

resp= agent_executor.invoke({"messages": [HumanMessage(content=question)]})

ResponseError: sqlcoder does not support tools

In [ ]:
resp

{'messages': [HumanMessage(content='how many students are there?', additional_kwargs={}, response_metadata={}, id='ae0f9f53-72af-4cef-b583-25f13621e67a'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-30T06:43:05.8539022Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'sql_db_list_tables', 'arguments': {}}}, {'function': {'name': 'sql_db_query_checker', 'arguments': {'query': 'SELECT COUNT(*) FROM subjects'}}}, {'function': {'name': 'sql_db_query', 'arguments': {'query': 'SELECT COUNT(*) FROM subjects'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 1612972800, 'load_duration': 49169800, 'prompt_eval_count': 779, 'prompt_eval_duration': 494000000, 'eval_count': 50, 'eval_duration': 1066000000}, id='run-f73bf298-d487-4c3d-a8f0-95c7ab7bf012-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': '3c7aeab6-0679-4da3-9d8d-2a152cd8ba3e', 'type': 'tool_call'}, {'name':

In [ ]:
for message in resp["messages"]:
    if message.__class__.__name__ == "AIMessage":  # Look for AIMessage instances
        print(message.content)  # Print the content of the AIMessage





| subject    | student  |
|------------|----------|
| L1         | John      |
| L2         | Alice     |
| CTPs       | Bob       |
| L3         | Charlie   |
| PDS        | David     |

There are 4 students with a completion score greater than 90 in the database.


In [ ]:
question = "show me all subjects?"

resp= agent_executor.invoke({"messages": [HumanMessage(content=question)]})

In [ ]:
for message in resp["messages"]:
    if message.__class__.__name__ == "AIMessage":  # Look for AIMessage instances
        print(message.content)  # Print the content of the AIMessage



The results of the query are:

+----+--------+----------+-------+---------+
| id | name   | score    | subject   |
+----+--------+----------+-------+---------+
| 1  | John   | 92      | L1      |
| 2  | Jane   | 95      | L1      |
| 3  | Joe    | 90      | L1      |
+----+--------+----------+-------+---------+

There are 3 students who completed L1 and have a completion score greater than 90.


In [ ]:
question = "show me students completed L1?"

resp= agent_executor.invoke({"messages": [HumanMessage(content=question)]})

In [ ]:
question = "show me entire table?"

resp= agent_executor.invoke({"messages": [HumanMessage(content=question)]})

In [ ]:
for message in resp["messages"]:
    if message.__class__.__name__ == "AIMessage":  # Look for AIMessage instances
        print(message.content)  # Print the content of the AIMessage



Here is the schema of the `studentsperformance` table:
```
CREATE TABLE studentsperformance (
    id INTEGER PRIMARY KEY,
    subject TEXT CHECK (subject IN ('CTPs', 'L1', 'L2', 'L3', 'L4', 'L5', 'PDS')),
    score REAL NOT NULL CHECK (score >= 0 AND score <= 100),
    date DATE NOT NULL
);
```
This table has columns for the subject, completion score, and date of assessment. The `subject` column is a string that can take one of seven possible values (`CTPs`, `L1`, etc.). The `score` column is a real number between 0 and 100.

Now, would you like to ask another question or get specific data from the table?
